# Constants

In [3]:
main_dir = "gpt2simple/"
drive_pre = "/content/drive/My Drive/"

# run_all: all the data
# run_1: only dataset 1 (15 melodies)
# run_2: only dataset 2 (other set of melodies)
run = "run_1"

# Encoding/Decoding

In [1]:
!pip install mido
from mido.messages.messages import Message as MidoMessage
from mido import MidiFile, MidiTrack, Message, MetaMessage
import shutil
import os
import tarfile

     |████████████████████████████████| 61kB 2.0MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Set up shared folder

gpt2simple: https://drive.google.com/drive/folders/1DJ_Dch1pS9LqtETvr7iIKbkLU5T1dWst?usp=sharing

1.   Open link
2.   Click on folder name in path, opens the dropdown
3.   Click on "Add shortcut to drive"
4.   You should now be able to access the model and training data

## Function implmentations

In [4]:
def encoder(midi_file, smallest_note=16):
  # Mido assumes 1 beat = 1/4 note, so convert to ticks per unit
  ticks_per_unit = ( midi_file.ticks_per_beat * 4 ) / smallest_note

  notes_to_play = []
  units = []
  current_unit = 0
  for msg in midi_file.tracks[0]:
    if type(msg) == MidoMessage:
      # How many units since the last message, msg.time is in ticks
      #   currently rounds to closest
      unit_delta = int(round(msg.time / ticks_per_unit))

      # Fill in all the time between then and now
      for i in range(current_unit,current_unit+unit_delta):
        if notes_to_play:
          units.append('/'.join(notes_to_play))
        else:
          units.append("-")

      # Do this after since msg.time is the # of ticks since last msg
      #   aka need to fill in the info since last and then remove/add the note
      if msg.type == "note_on":
        if str(msg.note) not in notes_to_play:
          notes_to_play.append(str(msg.note))
      elif msg.type == "note_off":
        if str(msg.note) in notes_to_play:
          notes_to_play.remove(str(msg.note))
      current_unit += unit_delta
  return " ".join(units) + "."

def decoder(midi_string, ticks_per_unit=120, name="Untitled track", instrument_name="Steinway Grand Piano", key="C", tempo=600000):
  new_mid = MidiFile()
  track = MidiTrack()
  new_mid.tracks.append(track)

  track.append(MetaMessage('channel_prefix', channel=0, time=0))
  track.append(MetaMessage('track_name', name=name, time=0))
  track.append(MetaMessage('instrument_name', name=instrument_name, time=0))
  track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
  track.append(MetaMessage('key_signature', key=key, time=0))
  track.append(MetaMessage('smpte_offset', frame_rate=24, hours=33, minutes=0, seconds=0, frames=0, sub_frames=0,  time=0))
  track.append(MetaMessage('set_tempo', tempo=tempo, time=0))

  playing_notes = []
  time_delta = 0
  for unit in midi_string.split():
    if unit == "":
      continue
    unit_string = unit[:]
    if unit[-1] == "," or unit[-1] == ".":
      unit_string = unit[:-1]
    if unit[0] == "-":
      continue
    notes = list(set([int(note) for note in unit_string.split("/")]))

    for playing_note in playing_notes:
      if playing_note not in notes:
        track.append(Message('note_off', note=playing_note, time=time_delta))
        time_delta=0

    for note in notes:
      if note not in playing_notes:
        track.append(Message('note_on', note=note, time=time_delta))
        time_delta=0
    
    time_delta += ticks_per_unit
    playing_notes = notes
  track.append(MetaMessage('end_of_track', time=len(midi_string.split()) * ticks_per_unit))
  return new_mid

## Encoding all the data
Run this before training

Only needed if training data has been changed/added

In [33]:
file_list = []
if run == "run_all" or run == "run_1":
  data_folder = drive_pre+"lofi_tracks_dataset/melody/"
  for item in os.listdir(data_folder):
    if item[-4:] == ".mid":
      file_list.append(data_folder+item)

if run == "run_all" or run == "run_2":
  data_folder2 = drive_pre+"lofi samples/melodies/midi/"
  for item in os.listdir(data_folder2):
    if item[-4:] == ".mid":
      file_list.append(data_folder2+item)

data_string = ""
for midi_file in file_list:
  print(">> Encoding", midi_file)
  mid_obj = MidiFile(midi_file)
  encoded_midi = encoder(mid_obj)
  if encoded_midi == ".":
    continue
  data_string += encoded_midi + "<|endoftext|>"

data_file_name = drive_pre+main_dir+"midi_text_dataset_{}.txt".format(run)

print("saving to",data_file_name)

with open(data_file_name, "w") as f:
  f.write(data_string)


>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_1_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_2_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_3_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_4_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_5_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_6_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_7_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_8_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_9_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack_10_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_dataset/melody/lofitrack11_melody.mid
>> Encoding /content/drive/My Drive/lofi_tracks_datas

## Decoding the data
Run this after generating

In [15]:
generated_data_folder = drive_pre + main_dir + "generated_midi_texts_{}/".format(run)
generated_file_list = []
for item in os.listdir(generated_data_folder):
  if item[-4:] == ".txt":
    generated_file_list.append(item)

generated_output_folder = drive_pre + main_dir + "generated_midi_output_{}/".format(run)
for midi_string_file in generated_file_list:
  try:
    save_file = generated_output_folder+midi_string_file[:-4]+".mid"
    if os.path.isfile(save_file):
      continue
    with open(generated_data_folder+midi_string_file) as f:
      mid_obj = decoder(f.read())
      mid_obj.save(save_file)
  except Exception as e:
    print("{} wasn't a valid endoded file".format(midi_string_file))
    print("    Error:", e)
    continue

midi_string_output_6_20200721_034854.txt wasn't a valid endoded file
    Error: invalid literal for int() with base 10: 'r'


#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [5]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [6]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 533Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 119Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 423Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 194Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 322Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 197Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 212Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [7]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory


In [9]:
file_name = "midi_text_dataset_{}.txt".format(run)

def copy_file_from_gdrive(file_name):
  gpt2.is_mounted()
  shutil.copyfile(drive_pre + main_dir + file_name, file_name)

copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [10]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='latest',
              run_name=run,
              print_every=10,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 1891.88it/s]

Loading dataset...
dataset has 29743 tokens
Training...


[10 | 18.06] loss=0.38 avg=0.38
[20 | 30.52] loss=0.31 avg=0.34
[30 | 42.98] loss=0.49 avg=0.39
[40 | 55.45] loss=0.37 avg=0.39
[50 | 67.93] loss=0.32 avg=0.37
[60 | 80.41] loss=0.38 avg=0.37
[70 | 92.87] loss=0.12 avg=0.34
[80 | 105.32] loss=0.13 avg=0.31
[90 | 117.79] loss=0.10 avg=0.28
[100 | 130.25] loss=0.12 avg=0.27
======== SAMPLE 1 ========
/63/68 53/56/59/62/70 53/56/59/62/70 53/56/59/62/70 53/56/59/62/70 53/56/59/62/70 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/68 50/58/60/63/

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [11]:
# gpt2.copy_checkpoint_to_gdrive(run_name='run1')
def copy_checkpoint_to_gdrive(run_name='run1'):
  gpt2.is_mounted()

  checkpoint_folder = os.path.join('checkpoint', run_name)

  file_path = gpt2.get_tarfile_name(checkpoint_folder)

  with tarfile.open(file_path, 'w') as tar:
    tar.add(checkpoint_folder)

  print("Copying to", drive_pre + main_dir + file_path)
  shutil.copyfile(file_path, drive_pre + main_dir + file_path)

copy_checkpoint_to_gdrive(run)

Copying to /content/drive/My Drive/gpt2simple/checkpoint_run_1.tar


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
#gpt2.copy_checkpoint_from_gdrive(run_name='run1')
def copy_checkpoint_from_gdrive(run_name='run1'):
  gpt2.is_mounted()

  checkpoint_folder = os.path.join('checkpoint', run_name)

  file_path = gpt2.get_tarfile_name(checkpoint_folder)

  shutil.copyfile(drive_pre + main_dir + + file_path, file_path)

  with tarfile.open(file_path, 'r') as tar:
    tar.extractall()
  
copy_checkpoint_from_gdrive(run)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [14]:
genned = gpt2.generate(sess, nsamples=10, run_name=run, return_as_list=True)

save_folder = drive_pre + main_dir + "generated_midi_texts_{}/".format(run)

i = 1
for gen in genned:
    for block in gen.split("<|endoftext|>"):
        for line in block.split("\n"):
            for song in line.split("."):
                if song == "":
                    continue
                if song[0] == "/":
                  song = song[1:]
                if song[-1] == "/":
                    song = song[:-1]
                song = song + "."
                file_name = "midi_string_output_{}_{:%Y%m%d_%H%M%S}.txt".format(i,datetime.utcnow())
                with open(save_folder+file_name, "w") as f:
                    f.write(song)
                i += 1

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.